# Reviews Classification with BERT
Bidirectional Encoder Representations from Transformers. BERT is a **text representation technique** which is a fusion of variety of state-of-the-art deep learning algorithms, such as bidirectional encoder LSTM and Transformers.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip -q /content/drive/MyDrive/hotel_reviews_dataset_train_arabic_balanced.zip -d data/

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [10]:
df = pd.read_csv('data/hotel_reviews_dataset_train_arabic_balanced.csv')
df.head()

,no,Hotel_name,Rating,User_type,Room_type,Nights,Review
0,2,فندق 72,2,مسافر منفرد,غرفة ديلوكس مزدوجة أو توأم,أقمت ليلة واحدة,“ممتاز”. النظافة والطاقم متعاون.
1,3,فندق 72,5,زوج,غرفة ديلوكس مزدوجة أو توأم,أقمت ليلة واحدة,استثنائي. سهولة إنهاء المعاملة في الاستقبال. ل...
2,16,فندق 72,5,زوج,-,أقمت ليلتين,استثنائي. انصح بأختيار الاسويت و بالاخص غرفه ر...
3,20,فندق 72,1,زوج,غرفة قياسية مزدوجة,أقمت ليلة واحدة,“استغرب تقييم الفندق كخمس نجوم”. لا شي. يستحق ...
4,23,فندق 72,4,زوج,غرفة ديلوكس مزدوجة أو توأم,أقمت ليلتين,جيد. المكان جميل وهاديء. كل شي جيد ونظيف بس كا...


### Using tensorflow

we will import **tensorflow_hub**, which basically is a place where you can find all the prebuilt and pretrained models developed in TensorFlo.

In [6]:
!pip install bert-for-tf2
!pip install sentencepiece
!pip install tensorflow-hub

     |████████████████████████████████| 51kB 7.7MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.7-cp36-none-any.whl size=30537 sha256=141582dd04a886e013d1e21c109e598da1c91c459018ff7ba83feed0c25b898f
  Stored in directory: /root/.cache/pip/wheels/e1/f8/e2/b98f79a6b8cc898d8e4102b83acb8a098df7d27500a2bac912
  Created wheel for py-params: filename=py_params-0.10.1-cp36-none-any.whl size=7849 sha256=d465121fd1aad846f304e10c457f0b3d9832dfd9fdcef3d402bf3467001e230a
  Stored in directory: /root/.cache/pip/wheels/fd/13/cf/731530f5760266e69a40217ea27fa0d39a2d2a67230a73e2bc
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19474 sha256=10dc6fc3cdf4d2c1a9695fcaf477a3282e014ffa244e0d628b698dfb2e380be1
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.2MB 13.1MB/s 


In [7]:
import tensorflow_hub as hub
import numpy as np
import bert
import random
import math

In [8]:
df['Rating'] = df['Rating'].apply(lambda x: 1 if x <= 3 else 0)
df.head(10)

,no,Hotel_name,Rating,User_type,Room_type,Nights,Review
0,2,فندق 72,1,مسافر منفرد,غرفة ديلوكس مزدوجة أو توأم,أقمت ليلة واحدة,“ممتاز”. النظافة والطاقم متعاون.
1,3,فندق 72,0,زوج,غرفة ديلوكس مزدوجة أو توأم,أقمت ليلة واحدة,استثنائي. سهولة إنهاء المعاملة في الاستقبال. ل...
2,16,فندق 72,0,زوج,-,أقمت ليلتين,استثنائي. انصح بأختيار الاسويت و بالاخص غرفه ر...
3,20,فندق 72,1,زوج,غرفة قياسية مزدوجة,أقمت ليلة واحدة,“استغرب تقييم الفندق كخمس نجوم”. لا شي. يستحق ...
4,23,فندق 72,0,زوج,غرفة ديلوكس مزدوجة أو توأم,أقمت ليلتين,جيد. المكان جميل وهاديء. كل شي جيد ونظيف بس كا...
5,24,فندق 72,0,أسرة,غرفة ديلوكس مزدوجة أو توأم,أقمت ليلة واحدة,ممتاز. موقع الفندق ونظافته والاطلاله على البحر...
6,25,فندق 72,0,زوج,غرفة ديلوكس مزدوجة أو توأم,أقمت ليلة واحدة,“جيدجداً”. الافطار جيد والسرير ممتاز ومريح واط...
7,26,فندق 72,0,مسافر منفرد,-,أقمت ليلتين,“فندق ممتاز”. الاثاث، النظافه.
8,27,فندق 72,0,أسرة,غرفة ديلوكس مزدوجة أو توأم,أقمت ليلتين,“الراحة و الهدوء”. مكان مناسب ومريح انصح به خ...
9,28,فندق 72,0,زوج,غرفة ديلوكس مزدوجة أو توأم,أقمت ليلة واحدة,استثنائي. المكان روعه تحديدا الغرف المطله على ...


In [9]:
df = df[['Review', 'Rating']]
df.head(10)

,Review,Rating
0,“ممتاز”. النظافة والطاقم متعاون.,1
1,استثنائي. سهولة إنهاء المعاملة في الاستقبال. ل...,0
2,استثنائي. انصح بأختيار الاسويت و بالاخص غرفه ر...,0
3,“استغرب تقييم الفندق كخمس نجوم”. لا شي. يستحق ...,1
4,جيد. المكان جميل وهاديء. كل شي جيد ونظيف بس كا...,0
5,ممتاز. موقع الفندق ونظافته والاطلاله على البحر...,0
6,“جيدجداً”. الافطار جيد والسرير ممتاز ومريح واط...,0
7,“فندق ممتاز”. الاثاث، النظافه.,0
8,“الراحة و الهدوء”. مكان مناسب ومريح انصح به خ...,0
9,استثنائي. المكان روعه تحديدا الغرف المطله على ...,0


**BERT MODEL**

In [6]:
import pickle
tokenizer = pickle.load(open('models/tokenizer.pkl', 'rb'))

In [7]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [11]:
tokenized_reviews = [tokenize_reviews(review) for review in df.Review]

In [16]:
import numpy as np
import random
y = np.array(df['Rating'])
# y = np.array(list(map(lambda x: 1 if x=="positive" else 0, y)))

In [17]:
reviews_with_len = [[review, y[i], len(review)]
                 for i, review in enumerate(tokenized_reviews)]

In [18]:
random.shuffle(reviews_with_len)

In [19]:
# sort the data by the length of the reviews and remove the length attribute from all the reviews.
reviews_with_len.sort(key=lambda x: x[2])
sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len] 

In [20]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))

In [21]:
BATCH_SIZE = 32 # after processing 32 reviews, the weights of the neural network will be updated...
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [23]:
import math
# split the dataset into subsets for training and testing...
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [3]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [ ]:
with tf.device('/device:GPU:0'):
    text_model = TEXT_MODEL(vocabulary_size = len(tokenizer.vocab),
                          embedding_dimensions = 200,
                          cnn_filters = 100,
                          dnn_units = 256,
                          model_output_classes = 2,
                          dropout_rate = 0.2)

    text_model.compile(loss="binary_crossentropy",
                            optimizer="adam",
                            metrics=["accuracy"])
  
    history = text_model.fit(train_data, epochs = 5)
    
    print("evaluation...")
    
    results = text_model.evaluate(test_data)

Epoch 1/5
2974/2974 [==============================] - 675s 224ms/step - loss: 0.1784 - accuracy: 0.9318
Epoch 2/5
2974/2974 [==============================] - 613s 206ms/step - loss: 0.1172 - accuracy: 0.9588
Epoch 3/5
2974/2974 [==============================] - 620s 208ms/step - loss: 0.0988 - accuracy: 0.9676
Epoch 4/5
2974/2974 [==============================] - 621s 208ms/step - loss: 0.0835 - accuracy: 0.9739
Epoch 5/5
2974/2974 [==============================] - 609s 204ms/step - loss: 0.0713 - accuracy: 0.9773
evaluation...
330/330 [==============================] - 2s 7ms/step - loss: 0.1545 - accuracy: 0.9662


In [32]:
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
def predict(texts):
    with tf.device('/device:CPU:0'):
        texts = pd.Series(texts)
        test_sequences = [tokenize_reviews(review) for review in texts]
        s = [len(x) for x in test_sequences]
        test_sequences = pad_sequences(test_sequences, maxlen=max(s), padding='post')
        text_model = TEXT_MODEL(vocabulary_size = len(tokenizer.vocab),
                              embedding_dimensions = 200,
                              cnn_filters = 100,
                              dnn_units = 256,
                              model_output_classes = 2,
                              dropout_rate = 0.2)
        
        text_model(test_sequences)

        text_model.load_weights('models/bertclf.h5')

        y_pred = text_model.predict(test_sequences)
        return [1 if p <= 0.5 else 0 for p in y_pred]

In [35]:
predict(['كانت سيئة جدًا'])

[0]